In [6]:
from pathlib import Path
try:
    import google.colab
    from google.colab import drive
    drive.mount('/content/drive', force_remount=False)
    PROJECT_ROOT = Path("/content/drive/MyDrive/ABT_Global/AI-Studio-Project")
except ImportError:
    PROJECT_ROOT = Path("../..").resolve()
PROCESSED = PROJECT_ROOT / "data" / "processed"
OUTPUTS = PROJECT_ROOT / "notebooks" / "outputs" / "milestone_2"
OUTPUTS.mkdir(parents=True, exist_ok=True)
print(f"Processed: {PROCESSED}")
print(f"Outputs: {OUTPUTS}")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Processed: /content/drive/MyDrive/ABT_Global/AI-Studio-Project/data/processed
Outputs: /content/drive/MyDrive/ABT_Global/AI-Studio-Project/notebooks/outputs/milestone_2


## Environment Setup


# 01 — Dimensionality Reduction

**Milestone 2** · Power Outage Prediction

This notebook performs dimensionality reduction on the **engineered features from Milestone 1**, selecting the most predictive subset for modeling. It uses univariate statistical methods (F-test, Mutual Information), correlation analysis, and optional PCA to identify the optimal feature set while preserving model performance.

**Pipeline:**
1. Load engineered features from Milestone 1 (no duplicate feature engineering)
2. Apply univariate selection methods (F-test, Mutual Information)
3. Remove highly correlated features (multicollinearity reduction)
4. Optional PCA analysis for dimensionality insights
5. Select final compact feature set for modeling

**Goal:** Reduce ~40+ engineered features to ~15 most predictive features, ensuring no data leakage and maintaining statistical rigor.


In [7]:
# === Setup ===
import json
import numpy as np
import pandas as pd

from sklearn.feature_selection import SelectKBest, f_classif, mutual_info_classif
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.decomposition import PCA

import warnings
warnings.filterwarnings("ignore")

## Load data

In [8]:

# Load engineered features from Milestone 1 (04_feature_engineering_selection.ipynb)
engineered_features_path = PROCESSED / "engineered_features.csv"

if engineered_features_path.exists():
    df = pd.read_csv(engineered_features_path)
    if 'date' in df.columns:
        df['date'] = pd.to_datetime(df['date'])
    source = "engineered_features.csv (from Milestone 1)"
else:
    raise FileNotFoundError(f"Engineered features not found! Please run Milestone 1 notebook (04_feature_engineering_selection.ipynb) first. Expected: {engineered_features_path}")

print(f"Loaded: {source} with shape {df.shape}")
print(f"Total features available: {len(df.columns)}")
print("Sample columns:", list(df.columns)[:15], "...")


Loaded: engineered_features.csv (from Milestone 1) with shape (3976135, 48)
Total features available: 48
Sample columns: ['fips_code', 'date', 'prcp', 'tmax', 'tmin', 'outage_occurred', 'customers_out', 'county', 'state', 'run_start_time', 'year', 'month', 'day_of_week', 'day_name', 'month_name'] ...


## Define target and exclude leakage / IDs

In [9]:

# Target
if 'outage_occurred' not in df.columns:
    raise KeyError("Column 'outage_occurred' is required as the classification target.")

y = df['outage_occurred'].astype(int)

# Columns to exclude from feature selection
# These were already identified and removed in Milestone 1 feature engineering
always_exclude = {
    # IDs / metadata
    'fips_code', 'county', 'state', 'region', 'date', 'county_name',
    # Leakage features (identified in Milestone 1)
    'run_start_time', 'customers_out', 'outage_hour',
    'state_risk_score', 'region_risk_score', 'state_risk_category', 'high_risk_state',
    # Temporal features excluded in Milestone 1
    'year',
    # Redundant features (keep better versions)
    'day_of_week', 'tmin_bin', 'tmax_bin', 'month_group',
    'day_of_year_sin', 'month_sin', 'day_of_week_cos',
    # Text labels
    'day_name', 'season'
}

exclude_cols = [c for c in df.columns if c in always_exclude or c == 'outage_occurred']

X = df.drop(columns=[c for c in exclude_cols if c in df.columns]).copy()
print(f"Features after excluding leakage/IDs: {X.shape[1]}")
print(f"Excluded {len([c for c in exclude_cols if c in df.columns])} columns")


Features after excluding leakage/IDs: 25
Excluded 23 columns


## Preprocess: encode categoricals and impute missing

In [12]:

# Features were already encoded in Milestone 1, but check for any remaining categorical
cat_cols = X.select_dtypes(include=['object','category']).columns.tolist()
if cat_cols:
    print(f"Encoding {len(cat_cols)} remaining categorical columns: {cat_cols}")
    for col in cat_cols:
        le = LabelEncoder()
        X[col] = le.fit_transform(X[col].astype(str))
else:
    print("All features are already numeric (encoded in Milestone 1)")

# Check and handle missing values
missing_summary = X.isnull().sum()
features_with_missing = missing_summary[missing_summary > 0]

if len(features_with_missing) > 0:
    print(f"\nHandling missing values in {len(features_with_missing)} features:")
    to_drop_100_missing = []
    for col, count in features_with_missing.items():
        print(f"  {col}: {count} missing ({count/len(X)*100:.2f}%)")
        if count == len(X): # If 100% missing
            to_drop_100_missing.append(col)
        elif pd.api.types.is_numeric_dtype(X[col]):
            X[col] = X[col].fillna(X[col].median())
        else:
            X[col] = X[col].fillna(X[col].mode().iloc[0])

    if to_drop_100_missing:
        print(f"\nDropping columns with 100% missing values: {to_drop_100_missing}")
        X = X.drop(columns=to_drop_100_missing)
else:
    print("No missing values found")

print(f"\nFinal preprocessed shape: {X.shape}")
print(f"Ready for dimensionality reduction with {X.shape[1]} features")


All features are already numeric (encoded in Milestone 1)

Handling missing values in 1 features:
  season_encoded: 3976135 missing (100.00%)

Dropping columns with 100% missing values: ['season_encoded']

Final preprocessed shape: (3976135, 24)
Ready for dimensionality reduction with 24 features


## Univariate selection: F-test and Mutual Information

In [13]:

# Dimensionality reduction: select top K features using univariate methods
K = 25  # target number of features to select

print(f"Running univariate feature selection on {X.shape[1]} features...")
print(f"Target: top {K} features from each method\n")

# F-test (ANOVA) - measures linear dependency with target
print("1. Computing F-statistics...")
f_selector = SelectKBest(score_func=f_classif, k=min(K, X.shape[1]))
f_selector.fit(X, y)
f_scores = f_selector.scores_
f_rank = pd.Series(f_scores, index=X.columns).sort_values(ascending=False).rename("f_score")

# Mutual Information - captures nonlinear relationships
print("2. Computing Mutual Information scores...")
mi_selector = SelectKBest(score_func=mutual_info_classif, k=min(K, X.shape[1]))
mi_selector.fit(X, y)
mi_scores = mi_selector.scores_
mi_rank = pd.Series(mi_scores, index=X.columns).sort_values(ascending=False).rename("mi_score")

# Combine rankings
rank_df = pd.concat([f_rank, mi_rank], axis=1).fillna(0)
rank_df['f_rank'] = rank_df['f_score'].rank(ascending=False)
rank_df['mi_rank'] = rank_df['mi_score'].rank(ascending=False)
rank_df['avg_rank'] = (rank_df['f_rank'] + rank_df['mi_rank']) / 2
rank_df = rank_df.sort_values('avg_rank', ascending=True)

# Save detailed scores
rank_df.to_csv(PROCESSED / "feature_scores_univariate.csv", index=True)
print(f"✓ Saved univariate scores -> {PROCESSED / 'feature_scores_univariate.csv'}")

# Select top features from each method
top_f  = f_rank.head(K).index.tolist()
top_mi = mi_rank.head(K).index.tolist()

print(f"\nTop {K} features by F-statistic: {len(top_f)}")
print(f"Top {K} features by Mutual Info: {len(top_mi)}")

# Create candidate pools
cand_union = list(dict.fromkeys(top_f + top_mi))  # union, preserving order
cand_inter = [c for c in top_f if c in top_mi]    # intersection

print(f"\n Candidate pools:")
print(f"   Union (F ∪ MI): {len(cand_union)} features")
print(f"   Intersection (F ∩ MI): {len(cand_inter)} features")


Running univariate feature selection on 24 features...
Target: top 25 features from each method

1. Computing F-statistics...
2. Computing Mutual Information scores...
✓ Saved univariate scores -> /content/drive/MyDrive/ABT_Global/AI-Studio-Project/data/processed/feature_scores_univariate.csv

Top 25 features by F-statistic: 24
Top 25 features by Mutual Info: 24

 Candidate pools:
   Union (F ∪ MI): 24 features
   Intersection (F ∩ MI): 24 features


## Correlation pruning (remove highly collinear features)

In [14]:

def correlation_prune(cols, corr_threshold=0.85):
    """
    Remove highly correlated features to reduce multicollinearity.
    Keeps the first feature in each correlated pair.
    """
    if len(cols) <= 1:
        return cols

    C = X[cols].corr().abs()
    # Upper triangle mask to avoid duplicates
    upper = C.where(np.triu(np.ones(C.shape), k=1).astype(bool))

    to_drop = set()
    dropped_pairs = []

    for c in upper.columns:
        if c in to_drop:
            continue
        high_corr = upper[c][upper[c] > corr_threshold]
        if not high_corr.empty:
            for corr_feat in high_corr.index:
                dropped_pairs.append((c, corr_feat, upper.loc[corr_feat, c]))
                to_drop.add(corr_feat)

    kept = [c for c in cols if c not in to_drop]

    if dropped_pairs:
        print(f"   Removed {len(to_drop)} highly correlated features (|r| > {corr_threshold}):")
        for feat1, feat2, corr in dropped_pairs[:5]:  # show first 5
            print(f"     • {feat2} (r={corr:.3f} with {feat1})")
        if len(dropped_pairs) > 5:
            print(f"     ... and {len(dropped_pairs) - 5} more pairs")

    return kept

print("\nCorrelation pruning (removing multicollinear features)...")

# Prune both candidate pools
union_pruned = correlation_prune(cand_union, corr_threshold=0.85)
inter_pruned = correlation_prune(cand_inter, corr_threshold=0.85)

print(f"\nAfter correlation pruning:")
print(f"   Union: {len(cand_union)} → {len(union_pruned)} features")
print(f"   Intersection: {len(cand_inter)} → {len(inter_pruned)} features")



Correlation pruning (removing multicollinear features)...
   Removed 5 highly correlated features (|r| > 0.85):
     • month_cos (r=0.955 with day_of_year_cos)
     • tmin (r=0.982 with temp_avg)
     • tmin (r=0.935 with tmax)
     • temp_avg (r=0.985 with tmax)
     • day_of_year (r=0.997 with month)
     ... and 1 more pairs
   Removed 5 highly correlated features (|r| > 0.85):
     • month_cos (r=0.955 with day_of_year_cos)
     • tmin (r=0.982 with temp_avg)
     • tmin (r=0.935 with tmax)
     • temp_avg (r=0.985 with tmax)
     • day_of_year (r=0.997 with month)
     ... and 1 more pairs

After correlation pruning:
   Union: 24 → 19 features
   Intersection: 24 → 19 features


## Optional PCA on continuous features (for reference)

In [15]:

# We'll only run PCA on numeric columns as an auxiliary representation.
# Save explained-variance info; do not force components into final feature set by default.
num_cols = X.select_dtypes(include=[np.number]).columns.tolist()
if len(num_cols) >= 2:
    scaler = StandardScaler()
    Xs = scaler.fit_transform(X[num_cols])
    pca = PCA(n_components=None, random_state=42)
    pca.fit(Xs)
    evr = pca.explained_variance_ratio_
    cum_evr = np.cumsum(evr)

    # Choose #components to reach >=95% variance
    n95 = int(np.searchsorted(cum_evr, 0.95) + 1)
    pca_info = pd.DataFrame({
        "component": np.arange(1, len(evr)+1),
        "explained_variance_ratio": evr,
        "cumulative_evr": cum_evr
    })
    pca_info.to_csv(PROCESSED / "pca_explained_variance.csv", index=False)
    print(f"PCA numeric cols: {len(num_cols)} | components to reach 95% var: {n95}")
    print("Saved PCA EVR ->", PROCESSED / "pca_explained_variance.csv")
else:
    print("Not enough numeric columns for PCA (skipped).")


PCA numeric cols: 24 | components to reach 95% var: 13
Saved PCA EVR -> /content/drive/MyDrive/ABT_Global/AI-Studio-Project/data/processed/pca_explained_variance.csv


## Final feature selection (rule-based consolidation)

In [17]:

# Final feature selection strategy:
# 1. Start with intersection (features strong in both methods)
# 2. Add top union features until we reach target size
# 3. Prioritize features by average rank

TARGET_K = 15  # final number of features for modeling

print(f"\nFinal feature selection (target: {TARGET_K} features)...")

# Start with intersection (most robust)
final_feats = inter_pruned.copy()

# Add from union if needed, sorted by average rank
if len(final_feats) < TARGET_K:
    # Get remaining union features, sorted by average rank
    remaining = [f for f in union_pruned if f not in final_feats]
    remaining_ranks = rank_df.loc[remaining, 'avg_rank'].sort_values()

    for feat in remaining_ranks.index:
        if len(final_feats) >= TARGET_K:
            break
        final_feats.append(feat)

print(f"\nSelected {len(final_feats)} features:")
print(f"   • {len(inter_pruned)} from intersection (robust across both methods)")
print(f"   • {len(final_feats) - len(inter_pruned)} additional from union")

# Display final features with their scores
print("\nFinal Feature Set (ranked by importance):")
final_rank_info = rank_df.loc[final_feats].sort_values('avg_rank')
for i, (feat, row) in enumerate(final_rank_info.iterrows(), 1):
    print(f"{i:2d}. {feat:<25} F={row['f_score']:>8.1f}  MI={row['mi_score']:.4f}")

# Save final features
with open(PROCESSED / "final_features.json", "w") as f:
    json.dump(final_feats, f, indent=2)
print(f"\n💾 Saved final features -> {PROCESSED / 'final_features.json'}")



Final feature selection (target: 15 features)...

Selected 19 features:
   • 19 from intersection (robust across both methods)
   • 0 additional from union

Final Feature Set (ranked by importance):
 1. is_weekend                F=  8166.1  MI=0.0865
 2. day_of_week_sin           F=  5701.8  MI=0.0702
 3. state_encoded             F=  3890.0  MI=0.0859
 4. region_encoded            F=   854.0  MI=0.1155
 5. prcp_category             F=    55.6  MI=0.1593
 6. has_precipitation         F=    46.7  MI=0.1346
 7. day_of_year_cos           F=  1497.4  MI=0.0033
 8. extreme_cold              F=   521.9  MI=0.0070
 9. month                     F=   149.4  MI=0.0407
10. temp_stress               F=    96.6  MI=0.0134
11. day                       F=    64.8  MI=0.0144
12. month_name                F=    46.1  MI=0.0427
13. tmax                      F=  1083.6  MI=0.0006
14. extreme_hot               F=    86.4  MI=0.0066
15. temp_range                F=   256.1  MI=0.0009
16. heavy_precipitat

## Save reduced dataset

In [18]:
# Create reduced dataset with selected features
reduced = X[final_feats].copy()
reduced['outage_occurred'] = y.values

# Keep metadata columns for analysis/evaluation
metadata_cols = ['date', 'fips_code', 'state', 'county']
for col in metadata_cols:
    if col in df.columns and col not in reduced.columns:
        reduced[col] = df[col]

# Save as both CSV and Parquet
reduced_csv = PROCESSED / "reduced_dataset.csv"
reduced_parquet = PROCESSED / "reduced_dataset.parquet"

reduced.to_csv(reduced_csv, index=False)
reduced.to_parquet(reduced_parquet, index=False)

print(f"\n Saved reduced dataset:")
print(f"   CSV: {reduced_csv}")
print(f"   Parquet: {reduced_parquet}")
print(f"   Shape: {reduced.shape}")
print(f"   Features: {len(final_feats)} + target + {len([c for c in metadata_cols if c in reduced.columns])} metadata columns")



 Saved reduced dataset:
   CSV: /content/drive/MyDrive/ABT_Global/AI-Studio-Project/data/processed/reduced_dataset.csv
   Parquet: /content/drive/MyDrive/ABT_Global/AI-Studio-Project/data/processed/reduced_dataset.parquet
   Shape: (3976135, 24)
   Features: 19 + target + 4 metadata columns
